# Bündelblockausgleichung

Mit Aruco-Markern und Näherungskoordinaten aus GNSS

In [1]:
from PIL import Image
from glob import glob
import cv2
from cv2 import aruco
import numpy as np
from sympy import *
import math
from PIL import ExifTags
from xml.etree import ElementTree as ET
import utm
import math

## Parameter festlegen
Sensorgröße in Millimetern und Pixeln

In [2]:
Xsensor = 0.00617
Ysensor = 0.00455
Xpixel = 4000.
Ypixel = 3000.

In [3]:
bilder = glob('./bildverband2/*.JPG')[0:2]

bilderDaten = []

for bild in bilder:
    bildDaten = []
    img = Image.open(bild)
    exif = img._getexif()
    for tag, value in exif.items():
        if 'GPSInfo' == ExifTags.TAGS.get(tag, tag):
            n = float(value[2][0])+float(value[2][1]) / \
                60.+float(value[2][2])/3600.
            e = float(value[4][0])+float(value[4][1]) / \
                60.+float(value[4][2])/3600.
            h = float(value[6])
            u = utm.from_latlon(n,e, 32)
            bildDaten.append(u[0])
            bildDaten.append(u[1])
            bildDaten.append(h)
    with open(bild, "rb") as f:
        s = str(f.read())

    start = s.find('<x:xmpmeta')
    end = s.find('</x:xmpmeta')
    xmp = s[start:end+12].replace("\\n", "\n")
    tree = ET.XML(xmp)

    bildDaten.append(float(tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightRollDegree'])/180*math.pi)
   
    bildDaten.append(float(
        tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightPitchDegree'])/180*math.pi)
    bildDaten.append(float(
        tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}FlightYawDegree'])/180*math.pi)
    #bildDaten.append(float(tree[0][0].attrib['{http://www.dji.com/drone-dji/1.0/}RelativeAltitude'])/180*math.pi)
    bilderDaten.append(bildDaten)
bilderDaten = np.array(bilderDaten)


## Marker identifizieren

In [4]:

LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

aruco_dict = aruco.Dictionary_create(32, 3)
beobachtungen = []

markers = set()
for bildnr in range(len(bilder)):
    bild = bilder[bildnr]
    cv_img = cv2.imread(bild)
    tmp = cv2.LUT(cv_img, lut)
    gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    
    parameter = aruco.DetectorParameters.create()
    parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX;
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=parameter)

    for nr in range(len(ids)):
        for cid in range(len(corners[nr][0])):
            markers.add(ids[nr][0]*10 + cid)
            beobachtungen.append(
                (bildnr, ids[nr][0]*10 + cid, 0, (corners[nr][0][cid][0]/Xpixel-0.5)*Xsensor))
            beobachtungen.append(
                    (bildnr, ids[nr][0]*10 + cid, 1, (corners[nr][0][cid][1]/Ypixel-0.5)*Ysensor))

    #marked = aruco.drawDetectedMarkers(cv_img, corners, ids)
    #cv2.imshow('image', cv2.resize(marked, (800,600)))
    # cv2.waitKey(0)

## Vektor mit Beobachtungen
Erstmal nur Marker aus den Bildern, wird noch mit weiteren Daten gefüllt

In [5]:

l = [i[len(i)-1] for i in beobachtungen]


## Festlegen von Symbolen
für die Unbekannten

In [6]:
sym = []

innereOrientierung = [
    Symbol('c'),
    Symbol('deltax'),
    Symbol('deltay')
]

sym.extend(innereOrientierung)

bildOrientierung = []
for bild in range(len(bilder)):
    bildO = []
    bildO.append(Symbol('bild'+str(bild)+'_x'))
    bildO.append(Symbol('bild'+str(bild)+'_y'))
    bildO.append(Symbol('bild'+str(bild)+'_z'))
    bildO.append(Symbol('bild'+str(bild)+'_kappa'))
    bildO.append(Symbol('bild'+str(bild)+'_omega'))
    bildO.append(Symbol('bild'+str(bild)+'_phi'))
    bildOrientierung.append(bildO)
    sym.extend(bildO)

neupunkte = []
for marker in list(markers):
    neup = []
    neup.append(Symbol('marker'+str(marker)+'x'))
    neup.append(Symbol('marker'+str(marker)+'y'))
    neup.append(Symbol('marker'+str(marker)+'z'))
    neupunkte.append(neup)
    sym.extend(neup)

print(sym)


[c, deltax, deltay, bild0_x, bild0_y, bild0_z, bild0_kappa, bild0_omega, bild0_phi, bild1_x, bild1_y, bild1_z, bild1_kappa, bild1_omega, bild1_phi, marker0x, marker0y, marker0z, marker1x, marker1y, marker1z, marker2x, marker2y, marker2z, marker3x, marker3y, marker3z, marker10x, marker10y, marker10z, marker11x, marker11y, marker11z, marker12x, marker12y, marker12z, marker13x, marker13y, marker13z, marker140x, marker140y, marker140z, marker141x, marker141y, marker141z, marker142x, marker142y, marker142z, marker143x, marker143y, marker143z, marker160x, marker160y, marker160z, marker161x, marker161y, marker161z, marker162x, marker162y, marker162z, marker163x, marker163y, marker163z, marker180x, marker180y, marker180z, marker181x, marker181y, marker181z, marker182x, marker182y, marker182z, marker183x, marker183y, marker183z]


## Erzeugen von Näherungswerten
- Brennweite: EXIF
- Verschiebung Fokuspunkt: 0
- Bilder: GNSS aus EXIF, Ausrichtung aus Kompass, Neigung 0
- Marker: Koordinaten des ersten Bildes plus Zufall

In [7]:
from random import random

x0 = []

bilderDatenFlat = bilderDaten.flatten()

for nr, __ in enumerate(sym):
    if nr == 0:  # Brennweite
        x0.append(0.0045)
    elif nr == 1:  # Fokuspunkt
        x0.append(0)
    elif nr == 2:  # Fokuspunkt
        x0.append(0)
    elif nr > 2 and nr < len(bilderDatenFlat)+3:
        x0.append(bilderDatenFlat[nr-3])
    elif (nr-3) % 3 == 0:
        x0.append(bilderDatenFlat[0]+(random()-5)*10)
    elif (nr-3) % 3 == 1:
        x0.append(bilderDatenFlat[1]+(random()-5)*10)
    elif (nr-3) % 3 == 2:
        x0.append(bilderDatenFlat[2]+(random()-5)*10)
    else:
        x0.append(0)


## Formeln
siehe Seite 281 Nahbereichsphotogrammetrie

folgende Formeln aus Nahbereichsphotogrammetrie S. 274

In [8]:
formeln = []


def r11(kappa, omega, phi):
    return cos(phi)*cos(kappa)-sin(phi)*sin(omega)*sin(kappa)


def r21(kappa, omega, phi):
    return sin(phi)*cos(kappa)+cos(phi)*sin(omega)*sin(kappa)


def r31(kappa, omega, phi):
    return -cos(omega)*sin(kappa)


def r12(kappa, omega, phi):
    return -sin(phi)*cos(omega)


def r22(kappa, omega, phi):
    return cos(phi)*cos(omega)


def r32(kappa, omega, phi):
    return sin(omega)


def r13(kappa, omega, phi):
    return cos(phi)*sin(kappa)+sin(phi)*sin(omega)*cos(kappa)


def r23(kappa, omega, phi):
    return sin(phi)*sin(kappa)-cos(phi)*sin(omega)*cos(kappa)


def r33(kappa, omega, phi):
    return cos(omega)*cos(kappa)


def kx(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r11(kappa, omega, phi)*(x-x0)+r21(kappa, omega, phi)*(y-y0)+r31(kappa, omega, phi)*(z-z0)


def ky(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r12(kappa, omega, phi)*(x-x0)+r22(kappa, omega, phi)*(y-y0)+r32(kappa, omega, phi)*(z-z0)


def nenner(x, x0, y, y0, z, z0, kappa, omega, phi):
    return r13(kappa, omega, phi)*(x-x0)+r23(kappa, omega, phi)*(y-y0)+r33(kappa, omega, phi)*(z-z0)


def x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax):
    return x0+c*kx(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltax


def y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay):
    return y0+c*ky(x, x0, y, y0, z, z0, kappa, omega, phi)/nenner(x, x0, y, y0, z, z0, kappa, omega, phi) + deltay


def kolinear(xy, c, x0, y0, z0, kappa, omega, phi, x, y, z, deltax, deltay):
    if xy == 0:
        return x_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltax)
    return y_kolinear(c, x, x0, y, y0, z, z0, kappa, omega, phi, deltay)

In [9]:
formeln = []
for bnr in range(len(beobachtungen)):
    bildnr, punkt, xy, koord = beobachtungen[bnr]
    ori = bildOrientierung[bildnr]
    bo = bildOrientierung[bildnr]
    neup = neupunkte[list(markers).index(punkt)]
    formeln.append(kolinear(
        xy, innereOrientierung[0], bo[0], bo[1], bo[2], bo[3], bo[4], bo[5], neup[0], neup[1], neup[2], innereOrientierung[1],innereOrientierung[2]))


formeln.append(innereOrientierung[0])
l.append(x0[0])

for bild in range(len(bildOrientierung)):
    for bildP in range(3):
        formeln.append(bildOrientierung[bild][bildP])
        l.append(bilderDaten[bild][bildP])


## Bilden der A-Matrix-Vorlage
aus Ableitungen der Formeln

In [10]:
A_vorlage = []
for f in range(len(formeln)):
    zeile = []
    for s in range(len(sym)):
        formel = formeln[f].diff(sym[s])
        zeile.append(formel)
    A_vorlage.append(zeile)


## Gewichtung
erstmal alles gleich

In [11]:

P = np.eye(len(l))
print('P', P)


P [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


## Ab hier Schleife...

In [15]:
werte = {}

for nr in range(len(sym)):
    werte[sym[nr]] = x0[nr]

#print(werte)

A = np.empty((len(formeln), len(sym)))
for f in range(len(formeln)):
    for s in range(len(sym)):
        A[f][s] = A_vorlage[f][s].evalf(subs=werte)


l0 = np.empty(len(l))
for i in range(len(l0)):
    l0[i] = formeln[i].evalf(subs=werte)

dl = l - l0

N = np.transpose(A)@P@A
h = np.transpose(A)@P@dl
Qx = np.linalg.inv(N)
dx = Qx@h

x0 = x0 - dx

#print('x0', x0)

dx.sum()


9.305094104627302e+57

In [13]:
dx

array([-3.32078949e+07, -1.62100168e+09,  6.34504160e+08, -2.54970449e+07,
        3.48671261e+07,  2.90533020e+03,  3.31165467e+11,  5.82890815e+11,
       -5.93876615e+11,  3.43152289e+07, -8.09316354e+06,  3.50418356e+03,
       -8.98881290e+10, -6.92227932e+10,  1.74367146e+11, -1.25148046e+15,
       -1.17206919e+15, -1.07481127e+15, -1.30012579e+15, -1.28563779e+15,
       -1.34743539e+15, -1.40641201e+15, -1.16494975e+15, -1.14759808e+15,
       -1.36065047e+15, -1.04808959e+15, -1.25470168e+15, -1.12938205e+15,
       -1.12684641e+15, -1.03153089e+15, -1.26015119e+15, -1.20546686e+15,
       -1.21995217e+15, -1.43063589e+15, -1.19105295e+15, -1.34417659e+15,
       -1.28677059e+15, -9.96026538e+14, -1.21501404e+15,  7.12756068e+26,
        6.84089560e+26,  6.30667411e+26, -8.35313324e+26, -8.41037422e+26,
       -8.29166843e+26,  8.24321347e+26,  8.60803785e+26,  8.83083915e+26,
        1.16257244e+26,  1.16760317e+26,  1.08453646e+26, -4.50719219e+26,
       -4.77641458e+26, -